In [1]:
import os
import random
import glob
import scipy
import numpy as np
import pylab as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import pyplot as mp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.io import savemat, loadmat
from sklearn import datasets, linear_model, preprocessing
from scipy.stats import probplot, pearsonr
from sklearn.preprocessing import scale,robust_scale
from sklearn.metrics import mean_squared_error, r2_score
from scipy import interp
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

/Users/emilyhahn/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Configure environment

In [2]:
dACC_group=[]
L_dlPFC_group=[]
R_dlPFC_group=[]
L_IFG_group=[]
R_IFG_group=[]
conflict_group=[]
adapt_group=[]
rt_group=[]

dACC_group_hc=[]
L_dlPFC_group_hc=[]
R_dlPFC_group_hc=[]
L_IFG_group_hc=[]
R_IFG_group_hc=[]
conflict_group_hc=[]
adapt_group_hc=[]
rt_group_hc=[]

dACC_group_pts=[]
L_dlPFC_group_pts=[]
R_dlPFC_group_pts=[]
L_IFG_group_pts=[]
R_IFG_group_pts=[]
conflict_group_pts=[]
adapt_group_pts=[]
rt_group_pts=[]

regions=['dACC','L_dlPFC','R_dlPFC','L_IFG','R_IFG']
variables=['rt','adapt','conflict']
raw_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/msit_mri_behav'
preproc_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/completed'
LSS_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG'
LSS_estim_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG_estim'
censor_dir='/Users/emilyhahn/projects/msit_modeling/censor_data'
AIC_output='/Users/emilyhahn/projects/msit_modeling/AIC_output'

df=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_C=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I_C=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})

df_I_epoch_first45=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})
df_C_epoch_first45=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})

### Define Subjects

In [3]:
SUBJ_LIST = ['hc001','hc002','hc003','hc005','hc006','hc010','hc011','hc012','hc014',\
             'hc015','hc017','hc019','hc021','hc028','hc031','hc032','hc033','hc034',\
             'hc036','hc038','hc042','pp001','pp002','pp003','pp004','pp005','pp006',\
             'pp010','pp011','pp012','pp013','pp015','pp016']
# SUBJ_LIST = ['hc001']
# print len(SUBJ_LIST)

### Define Functions - Data Structs

In [4]:
total_outliers=[]
def remove_outliers(rt_var,RT_arr,sub):
    elements=[i for i in RT_arr]
    mean = np.nanmean(elements)
    sd = np.nanstd(elements)
    sd_lower = mean - 5 * sd
    sd_upper = mean + 5 * sd
    for x in elements:
        if (sd_lower <= x <= sd_upper):
            rt_var.append(x)
        else:
            rt_var.append('NaN')
            print '%s\n%.2f RT value excluded: not in range SD min (%.2f) to SD max (%.2f)' % (sub,x,sd_lower,sd_upper)
    return

def generate_roi_vars(roi_file,region,var,sub):
    elements=[]
    with open(roi_file, 'r') as file:
        for line in file:
            line=line.strip()
            line=float(line)
            elements.append(line)
    mean = np.nanmean(elements)
    sd = np.nanstd(elements)
    sd_lower = mean - 5 * sd
    sd_upper = mean + 5 * sd
    for x in elements:
        if (sd_lower <= x <= sd_upper):
            region.append(x)
        else:
            region.append('NaN')
            total_outliers.append(x)
            print '%s\n%.2f %s beta excluded: not in range SD min (%.2f) to SD max (%.2f)' % (sub,x,var,sd_lower,sd_upper) 
    return

def censor_tps(censor_file,censor_var,var):
    with open(censor_file, 'r') as file:
        for line in file:
            line=line.strip()
            censor_var.append(line)
    return

def build_plots(ROI,variable,df_type,group,cond,sub):
    slope, intercept, r_value, p_value, std_err = \
            stats.linregress(df_type['%s' % variable],df_type['%s' % ROI])
#     if (p_value <= 1):
    sns.set()
    g=sns.JointGrid(x='%s' % variable, y='%s' % ROI, data=df_type)
    g=g.plot(sns.regplot,sns.distplot)
    lin_reg_r = lambda a, b: stats.linregress(df_type['%s' % variable], df_type['%s' % ROI])[2:4]
    g = g.annotate(lin_reg_r, template="{stat}: {val:.2f} $p=$ {p:.2f}",stat="$r=$",loc="upper left")                                                                        
    plt.subplots_adjust(top=0.93)
#     g.fig.suptitle("%s | %s" % (group,cond))
    g.fig.suptitle("%s | %s | %s" % (group,cond,sub))
#     g.savefig('/Users/emilyhahn/projects/msit_modeling/figures/%s_%s_%s_%s.png' % (ROI,variable,group,cond))
    plt.show()
#     else:
#         print "**** %s %s %s not significant ****" % (SUBJ,ROI,cond)
    return

### Assemble Indiv and Group Data - SS output, ROI data
    This module generates DataFrames:
    df = ALL subjects, ALL trials
    df_I = ALL subjects, Incongruent trials
    df_C = ALL subjects, Congruent trials
    df_I_C =ALL subjects, Incongruent-Congruent trials
    df_hcs = CTRL subjects, ALL trials
    df_pts = PSYCH subjects, ALL trials
    df_hcs_I = CTRL subjects, Incongruent trials
    df_pts_I = PSYCH subjects, Incongruent trials
    df_hcs_C = CTRL subjects, Congruent trials
    df_pts_C = PSYCH subjects, Congruent trials

In [5]:
for SUBJ in SUBJ_LIST:
    rt=[]
    RT=[]
    RT_orig=[]
    conflict=[]
    adapt=[]
    dACC=[]
    L_IFG=[]
    R_IFG=[]
    L_dlPFC=[]
    R_dlPFC=[]
    cond=[]
    trial=[]
    acc=[]
    censor=[]        
    #---------------------------------##
    ## Configure SS variables
    #---------------------------------##
    mat = loadmat(os.path.join(preproc_behav_dir,'%s_msit_ss_iter250.mat') % SUBJ)
    #---------------------------------##
    ss_outputs_xsmt = np.expand_dims(np.array([np.concatenate(arr) \
                                for arr in mat['XSmt']]).squeeze(),1)
    ss_xsmt = ss_outputs_xsmt.squeeze()
    RT.extend([float(i) for i in np.array(mat['RT'].squeeze())])
    remove_outliers(rt,RT,SUBJ)
    #---------------------------------##
    conflict.extend([float(i[0]) for i in ss_xsmt])
    adapt.extend([float(i[1]) for i in ss_xsmt])
    acc.extend([float(i) for i in np.array(mat['Accuracy'].squeeze())])
    trial.extend([float(i) for i in np.array(mat['Trial'].squeeze())])
    cond.extend([float(i) for i in np.array(mat['Interference'].squeeze())])
    #---------------------------------##
    ## Configure ROI variables
    #---------------------------------##
    ## Load indiv ROI vars
    file_1=os.path.join(LSS_dir,'%s.dACC_LSS_avg_file.1D' % SUBJ)
    file_2=os.path.join(LSS_dir,'%s.L_IFG_LSS_avg_file.1D' % SUBJ)
    file_3=os.path.join(LSS_dir,'%s.R_IFG_LSS_avg_file.1D' % SUBJ)
    file_4=os.path.join(LSS_dir,'%s.L_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_5=os.path.join(LSS_dir,'%s.R_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_6=os.path.join(censor_dir,'msit.%s.msit_bsm.censor.1D' % SUBJ)
    #---------------------------------##
    ## Generate ROI variables
    generate_roi_vars(file_1,dACC,"dACC",SUBJ)
    generate_roi_vars(file_2,L_IFG,"L_IFG",SUBJ)
    generate_roi_vars(file_3,R_IFG,"R_IFG",SUBJ)
    generate_roi_vars(file_4,L_dlPFC,"L_dlPFC",SUBJ)
    generate_roi_vars(file_5,R_dlPFC,"R_dlPFC",SUBJ)
    #---------------------------------##
    censor_tps(file_6,censor,"censor")
    #---------------------------------##
    ## insert RT outliers scrubbing function here
    #---------------------------------##
    ## Determine group
    s=[]
    if 'hc' in SUBJ:
        s.append(0)
    elif 'pp' in SUBJ:
        s.append(1)
    GROUP_ARR=np.array(["%s" % s] * len(trial))
    group=[float(i.strip('[]')) for i in GROUP_ARR]
    SUBJ_ARR=np.array(["%s" % SUBJ] * len(trial))
    #---------------------------------## 
    ## Scale + interpolate missing ROI + RT data points
    ## Configure this section to statistically appropriate scaling
    
    dACC = pd.Series(dACC).astype(float).interpolate()
    R_IFG = pd.Series(R_IFG).astype(float).interpolate()
    L_IFG = pd.Series(L_IFG).astype(float).interpolate()
    L_dlPFC = pd.Series(L_dlPFC).astype(float).interpolate()
    R_dlPFC = pd.Series(R_dlPFC).astype(float).interpolate()

    rt = preprocessing.scale(pd.Series(rt).astype(float).interpolate())
    #---------------------------------## 
    dACC_group.append(dACC)
    L_dlPFC_group.append(L_dlPFC)
    R_dlPFC_group.append(R_dlPFC)
    L_IFG_group.append(L_IFG)
    R_IFG_group.append(R_IFG)
    conflict_group.append(conflict)
    adapt_group.append(adapt)
    rt_group.append(rt)
    #---------------------------------##
    if 'pp' in SUBJ_ARR[0]:
        dACC_group_pts.append(dACC)
        L_dlPFC_group_pts.append(L_dlPFC)
        R_dlPFC_group_pts.append(R_dlPFC)
        L_IFG_group_pts.append(L_IFG)
        R_IFG_group_pts.append(R_IFG)
        conflict_group_pts.append(conflict)
        adapt_group_pts.append(adapt)
        rt_group_pts.append(rt)
    elif 'hc' in SUBJ_ARR[0]:
        dACC_group_hc.append(dACC)
        L_dlPFC_group_hc.append(L_dlPFC)
        R_dlPFC_group_hc.append(R_dlPFC)
        L_IFG_group_hc.append(L_IFG)
        R_IFG_group_hc.append(R_IFG)
        conflict_group_hc.append(conflict)
        adapt_group_hc.append(adapt)  
        rt_group_hc.append(rt)
    else:
        "REVIEW SUBJ ID: %s" % SUBJ
    ##---------------------------------##
    ## Append subj data to master DF
    ##---------------------------------##
    df1=pd.DataFrame({"group":group,"subject":SUBJ_ARR,'rt':rt,'dACC':dACC,'L_IFG':L_IFG,\
                      'R_IFG':R_IFG,'L_dlPFC':L_dlPFC,'R_dlPFC':R_dlPFC,'trial':trial,\
                      'cond':cond,'acc':acc,'conflict':conflict,'adapt':adapt})
    
    ##---------------------------------##

    ##---------------------------------##
    ## Determine AIC, BIC
    ##---------------------------------##
    def OLS(equation,df_type,df_name,roi,mod):
        t_model = smf.ols(formula = '%s' % equation, data=df_type)
        t_lin_reg = t_model.fit()
        pvals = t_lin_reg.pvalues
        coeff = t_lin_reg.params
        stderr = t_lin_reg.bse
        AIC = t_lin_reg.aic
        conf_lower = t_lin_reg.conf_int(alpha=0.05,cols=None)[0]
        conf_higher = t_lin_reg.conf_int(alpha=0.05,cols=None)[1]
        results_df = pd.DataFrame(data={"model":equation,"subj":df_name,"p_vals":pvals,
                                        "β_coef":coeff,"std_err":stderr,"CI_low":conf_lower,
                                        "CI_high":conf_higher,"AIC":AIC})
        results_df.index.name = 'variable'
        results_df.reset_index(level=0, inplace=True)
        results_df = results_df[["subj","model","variable","AIC","β_coef","p_vals","std_err","CI_low","CI_high"]]  
        results_df.to_csv(os.path.join(AIC_output,'%s_%s_%s.csv' % (df_name,roi,mod)),sep=',',index=False)
        return
    ##---------------------------------##
    for cog_roi in regions:
        
        OLS('%s ~ 1' % cog_roi,df1,'%s' % SUBJ, cog_roi, '1')
        OLS('%s ~ cond + 1' % cog_roi,df1,'%s' % SUBJ, cog_roi, 'cond_1')
        OLS('%s ~ rt + 1' % cog_roi,df1,'%s' % SUBJ, cog_roi, 'rt_1')
        OLS('%s ~ adapt + 1' % cog_roi,df1,'%s' % SUBJ, cog_roi, 'adapt_1')
        OLS('%s ~ conflict + 1' % cog_roi,df1,'%s' % SUBJ, cog_roi,'conflict_1')
        OLS('%s ~ adapt + conflict + 1' % cog_roi,df1,'%s' % SUBJ, cog_roi,'adapt_conflict_1')
        
    ##---------------------------------##
    ##---------------------------------##
    df=df.append(df1)
    df1_C=df1[df1.cond == 0]
    df1_I=df1[df1.cond == 1]
    ##---------------------------------##
#     conditions = ['Incongruent','Congruent']
#     rois = ['dACC']

#     for c in conditions:
#         for region in rois:
#             for variable in variables:
#                 if 'pp' in SUBJ and c=='Incongruent':
#                     build_plots('%s' % region,'%s' % variable,df1_I,'PSYCH','%s' % c,SUBJ)
#                 elif 'pp' in SUBJ and c=='Congruent':
#                     build_plots('%s' % region,'%s' % variable,df1_C,'PSYCH','%s' % c,SUBJ)
#                 elif 'hc' in SUBJ and c=='Incongruent':
#                     build_plots('%s' % region,'%s' % variable,df1_I,'CTRL','%s' % c,SUBJ)
#                 elif 'hc' in SUBJ and c=='Congruent':
#                     build_plots('%s' % region,'%s' % variable,df1_C,'CTRL','%s' % c,SUBJ)
#                 else:
#                     print '%s %s %s %s' % (g,c,region,variable)
    ##---------------------------------##
    df1_C_epoch_first45=df1_C[:][0:50].reset_index(drop=True).mean(axis=0,numeric_only=True) 
    df1_I_epoch_first45=df1_I[:][0:50].reset_index(drop=True).mean(axis=0,numeric_only=True)
    ##---------------------------------##
    df1_C=df1_C.mean(axis=0,numeric_only=True)
    df1_I=df1_I.mean(axis=0,numeric_only=True)
    ##---------------------------------##
    ##---------------------------------##
    S=group[0]
    df1_C['subject']=SUBJ
    df1_C['group']=S
    df1_I['subject']=SUBJ
    df1_I['group']=S
    df1_I_epoch_first45['subject']=SUBJ
    df1_I_epoch_first45['group']=S
    df1_C_epoch_first45['subject']=SUBJ
    df1_C_epoch_first45['group']=S
    ##---------------------------------##
    df_I=df_I.append(df1_I,ignore_index=True)
    df_C=df_C.append(df1_C,ignore_index=True)
    
    df_I_epoch_first45=df_I_epoch_first45.append(df1_I_epoch_first45,ignore_index=True)
    df_C_epoch_first45=df_C_epoch_first45.append(df1_C_epoch_first45,ignore_index=True)
    ##---------------------------------##
    df1_I_C=pd.DataFrame(data={"group":S,"subject":SUBJ,'rt':df1_I['rt']-df1_C['rt']},index=[1])
    df1_I_C={"group":S,"subject":SUBJ,\
             'rt':df1_I['rt']-df1_C['rt'],\
             'dACC':df1_I['dACC']-df1_C['dACC'],\
             'L_IFG': df1_I['L_IFG']-df1_C['L_IFG'],\
             'R_IFG': df1_I['R_IFG']-df1_C['R_IFG'],\
             'L_dlPFC': df1_I['L_dlPFC']-df1_C['L_dlPFC'],\
             'R_dlPFC': df1_I['R_dlPFC']-df1_C['R_dlPFC'],\
             'acc': df1_I['acc']-df1_C['acc'],\
             'conflict': df1_I['conflict']-df1_C['conflict'],\
             'adapt': df1_I['adapt']-df1_C['adapt'] }
    df_I_C=df_I_C.append(df1_I_C,ignore_index=True)
##---------------------------------##
## Parse DataFrames
##---------------------------------##
df_hcs=df.where(df['group']==0).dropna()
df_pts=df.where(df['group']==1).dropna()
df_hcs_I=df_I.where(df_I['group']==0).dropna().reset_index(drop=True)
df_pts_I=df_I.where(df_I['group']==1).dropna().reset_index(drop=True)
df_hcs_C=df_C.where(df_C['group']==0).dropna().reset_index(drop=True)
df_pts_C=df_C.where(df_C['group']==1).dropna().reset_index(drop=True)
df_hcs_I_epoch_first45=df_I_epoch_first45.where(df_I_epoch_first45['group']==0).dropna().reset_index(drop=True)
df_hcs_C_epoch_first45=df_C_epoch_first45.where(df_C_epoch_first45['group']==0).dropna().reset_index(drop=True)
df_pts_I_epoch_first45=df_I_epoch_first45.where(df_I_epoch_first45['group']==1).dropna().reset_index(drop=True)
df_pts_C_epoch_first45=df_C_epoch_first45.where(df_C_epoch_first45['group']==1).dropna().reset_index(drop=True)

hc005
8.68 dACC beta excluded: not in range SD min (-4.03) to SD max (4.25)
hc006
nan RT value excluded: not in range SD min (-0.33) to SD max (2.45)
hc019
nan RT value excluded: not in range SD min (-0.02) to SD max (2.26)
hc019
nan RT value excluded: not in range SD min (-0.02) to SD max (2.26)
hc019
nan RT value excluded: not in range SD min (-0.02) to SD max (2.26)
hc019
nan RT value excluded: not in range SD min (-0.02) to SD max (2.26)
hc021
nan RT value excluded: not in range SD min (0.04) to SD max (2.09)
hc028
nan RT value excluded: not in range SD min (-0.45) to SD max (1.98)
hc031
nan RT value excluded: not in range SD min (-0.17) to SD max (1.79)
hc031
nan RT value excluded: not in range SD min (-0.17) to SD max (1.79)
hc031
nan RT value excluded: not in range SD min (-0.17) to SD max (1.79)
hc031
nan RT value excluded: not in range SD min (-0.17) to SD max (1.79)
hc031
nan RT value excluded: not in range SD min (-0.17) to SD max (1.79)
hc031
nan RT value excluded: not in r

### Combine all AIC .csv files

In [6]:
# all_files = glob.glob(AIC_output + "/*.csv")

# frame = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     frame.append(df)

# df_models = pd.concat(frame, axis=0, ignore_index=True)

# patient_status = []

# for sub in df_models['subj']:
#     if 'hc' in sub:
#         patient_status.append(0)
#     elif 'pp' in sub:
#         patient_status.append(1)
#     else:
#         print "check subj id"

# df_models['patient_status']=patient_status
# df_models = df_models.sort_values(by=['subj','model'])
# df_models=df_models.set_index(['subj','model','variable'])

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_models)

In [7]:
aic_decrease_file = glob.glob(AIC_output + "/AIC_plus_decrease_vals/*.csv")

frame_d = []

for filename_d in aic_decrease_file:
    df_d = pd.read_csv(filename_d, index_col=None, header=0)
    frame_d.append(df_d)

df_models_d = pd.concat(frame_d, axis=0, ignore_index=True)
df_models_d = df_models_d.sort_values(by=['model','variable'])
df_models_d = df_models_d.set_index(['model','variable'])

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_models_d)

subj          AIC  AIC_d  \
model                          variable                               
L_IFG ~ 1                      Intercept  hc001   182.287474    NaN   
                               Intercept  hc002   496.159903    NaN   
                               Intercept  hc003   108.973245    NaN   
                               Intercept  hc005   137.300358    NaN   
                               Intercept  hc006   437.829877    NaN   
                               Intercept  hc010   117.705158    NaN   
                               Intercept  hc011   384.534788    NaN   
                               Intercept  hc012   329.068522    NaN   
                               Intercept  hc014   301.161766    NaN   
                               Intercept  hc015  -137.161005    NaN   
                               Intercept  hc017   -94.224756    NaN   
                               Intercept  hc019   -99.643515    NaN   
                               Intercept  hc021   297.165307    NaN   
                               Intercept  hc028   -14.982206    NaN   
                               Intercept  hc031    91.688360    NaN   
                               Intercept  hc032  -105.222943    NaN   
                               Intercept  hc033    84.729644    NaN   
                               Intercept  hc034    34.123844    NaN   
                               Intercept  hc036  -293.257820    NaN   
                               Intercept  hc038   614.576428    NaN   
                               Intercept  hc042  -310.700533    NaN   
                               Intercept  pp001   186.649633    NaN   
                               Intercept  pp002   624.477900    NaN   
                               Intercept  pp003   179.188968    NaN   
                               Intercept  pp004   496.149588    NaN   
                               Intercept  pp005   313.020546    NaN   
                               Intercept  pp006   204.126022    NaN   
                               Intercept  pp010   503.007694    NaN   
                               Intercept  pp011   344.254130    NaN   
                               Intercept  pp012   -40.619832    NaN   
                               Intercept  pp013  -121.741770    NaN   
                               Intercept  pp015   272.114468    NaN   
                               Intercept  pp016   623.608416    NaN   
L_IFG ~ adapt + 1              Intercept  hc001   180.411315    1.0   
                               Intercept  hc002   497.958967    0.0   
                               Intercept  hc003   110.624836    0.0   
                               Intercept  hc005   138.289781    0.0   
                               Intercept  hc006   439.762550    0.0   
                               Intercept  hc010   119.397204    0.0   
                               Intercept  hc011   386.383703    0.0   
                               Intercept  hc012   331.051686    0.0   
                               Intercept  hc014   301.375742    0.0   
                               Intercept  hc015  -135.610229    0.0   
                               Intercept  hc017   -92.867751    0.0   
                               Intercept  hc019  -100.863825    1.0   
                               Intercept  hc021   298.888520    0.0   
                               Intercept  hc028   -13.079068    0.0   
                               Intercept  hc031    93.649738    0.0   
                               Intercept  hc032  -103.463344    0.0   
                               Intercept  hc033    85.627320    0.0   
                               Intercept  hc034    15.699885    1.0   
                               Intercept  hc036  -295.476489    1.0   
                               Intercept  hc038   614.860134    0.0   
                               Intercept  hc042  -309.575704    0.0   
                               Intercept  pp001   188.177969    0.0

## Random Forest Classifier

In [8]:
def Random_Forest_Classifier(ROI,X1,X2,binary_feat_X,binary_feat_y):

    np.random.seed(seed = 42)
    
    ## Assign X features - [rt, cond] [conflict, adapt], [conflict + adapt]
    X1_AIC_d = df_models_d.loc[('%s ~ %s + 1' % (ROI,X1),'%s' % X1),'%s' % binary_feat_X]
    X2_AIC_d = df_models_d.loc[('%s ~ %s + 1' % (ROI,X2),'%s' % X2),'%s' % binary_feat_X]
    
    ## Assign y label - patient status
    Y_pt_status = df_models_d.loc[('%s ~ %s + 1' % (ROI,X1),'%s' % X1),'%s' % binary_feat_y]

    X,X_features=np.stack((X1_AIC_d,X2_AIC_d),axis=1),['%s' % X1,'%s' % X2] 
    y=np.array(Y_pt_status)

    ## Split train/test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=5) 

    ## Gaussian classifier
    clf=RandomForestClassifier(n_estimators=100)

    ## Train model
    clf.fit(X_train,y_train)

    ## Test model
    y_pred=clf.predict(X_test)

    ## Model accuracy - how often is classifier correct?
    acc=metrics.accuracy_score(y_test, y_pred)

    ## Determine feature importance
    feature_imp = pd.Series(clf.feature_importances_,index=X_features).sort_values(ascending=False)
#     print feature_imp
    
    ## Plot feature importance
    sns.set()
    g=[]
    g=sns.barplot(x=feature_imp, y=feature_imp.index,
                palette=sns.diverging_palette(255, 10, n=2))
    g.set_xlim(0, 1)
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("%s | Accuracy: %.4f" % (ROI,acc))
    plt.figure()
    g = g.get_figure()
    pth = os.path.join(AIC_output,'AIC_plus_decrease_vals/figures/')
    g.savefig(os.path.join(pth,'%s_%s_%s_%s_%s.png' % 
                             (ROI,X1,X2,binary_feat_X,binary_feat_y)),
                dpi=g.dpi,bbox_inches='tight')
    return

for region in regions:
    Random_Forest_Classifier('%s' % region,'conflict','adapt','AIC_d','patient_status')
    Random_Forest_Classifier('%s' % region,'rt','cond','AIC_d','patient_status')

### Sanity check purposes only: Test an infinite amount of interactions

In [ ]:
# rt_pred_ROIs_group_cond='rt ~ cond + group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group_cond)
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(L_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group_cond)
# OLS(R_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group_cond)
# OLS(L_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group_cond)
# OLS(R_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group_cond)

# ##---------------------------------##
# rt_pred_ROIs_cond='rt ~ cond + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_cond='dACC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_cond='L_IFG ~ cond + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_cond='R_IFG ~ cond + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_cond='L_dlPFC ~ cond + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_cond='R_dlPFC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)
# ##---------------------------------##
# rt_pred_ROIs_group='rt ~ group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group='dACC ~ group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group='L_IFG ~ group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group='R_IFG ~ group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group='L_dlPFC ~ group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group='R_dlPFC ~ group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)
# ##---------------------------------##
# dACC_pred_ROIs_cond='dACC ~ cond + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_cond='L_IFG ~ cond + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_cond='R_IFG ~ cond + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_cond='L_dlPFC ~ cond + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_cond='R_dlPFC ~ cond + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group='dACC ~ group + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_group='L_IFG ~ group + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_group='R_IFG ~ group + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_group='L_dlPFC ~ group + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_group='R_dlPFC ~ group + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)
# ##---------------------------------##
# rt_pred_ROIs='rt ~ dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs='dACC ~ rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs='L_IFG ~ rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs='R_IFG ~ rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs='L_dlPFC ~ rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs='R_dlPFC ~ rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + rt + group'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt'

# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# rt_pred_group_cond='rt ~ cond + group'
# dACC_pred_group_cond='dACC ~ cond + group'
# L_IFG_pred_group_cond='L_IFG ~ cond + group'
# R_IFG_pred_group_cond='R_IFG ~ cond + group'
# L_dlPFC_pred_group_cond='L_dlPFC ~ cond + group'
# R_dlPFC_pred_group_cond='R_dlPFC ~ cond + group'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt='dACC ~ rt'
# L_IFG_pred_rt='L_IFG ~ rt'
# R_IFG_pred_rt='R_IFG ~ rt'
# L_dlPFC_pred_rt='L_dlPFC ~ rt'
# R_dlPFC_pred_rt='R_dlPFC ~ rt'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# rt_pred_cond='rt ~ cond'
# dACC_pred_cond='dACC ~ cond'
# L_IFG_pred_cond='L_IFG ~ cond'
# R_IFG_pred_cond='R_IFG ~ cond'
# L_dlPFC_pred_cond='L_dlPFC ~ cond'
# R_dlPFC_pred_cond='R_dlPFC ~ cond'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')

# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# ##---------------------------------##
# rt_pred_group='rt ~ group'
# dACC_pred_group='dACC ~ group'
# L_IFG_pred_group='L_IFG ~ group'
# R_IFG_pred_group='R_IFG ~ group'
# L_dlPFC_pred_group='L_dlPFC ~ group'
# R_dlPFC_pred_group='R_dlPFC ~ group'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# ##---------------------------------##